# 📊 Predicción de gastos recurrentes por cliente y comercio

Este análisis busca identificar patrones de gasto mensuales o bimestrales por cliente y comercio, para anticipar transacciones futuras en el mes más reciente disponible (ultimo_mes). Se realiza un filtrado de datos, agrupación, análisis de recurrencia y finalmente una validación para medir la precisión del modelo.

## 🧾 1. Lectura y preparación de los datos

Se importa el dataset con información de transacciones y se clasifica cada comercio en una categoría de gasto.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import calendar

In [ ]:
df_1  = pd.read_csv('base_clientes_final.csv')
df_2 = pd.read_csv('base_transacciones_final_SINtypeos.csv')
df = pd.merge(df_1, df_2, how='outer', on='id')
#df.to_csv('bases_merged.csv', index=False)

In [ ]:
#df =pd.read_csv("bases_merged.csv", sep=",", encoding="utf-8")
categoria_gasto = {
    "Movilidad y transporte": ["UBER", "DIDI", "UBRPAGOSMEX", "METROBUS", "RENTAMOVISTAR", "TOTAL PASS", "OXXO GAS"],
    "Comida y bebida": ["STARBUCKS", "OXXO", "WALMART", "CARLS JR", "HEB", "SUPERAMA", "CHEDRAUI", "RAPPI", "UBER EATS", "DIDI FOOD", "7 ELEVEN"],
    "Entretenimiento y streaming": ["NETFLIX", "SPOTIFY", "DISNEY PLUS", "VIX", "ROKU", "PLAYSTATION NETWORK", "ITUNES", "AUDIBLE", "CRUNCHYROLL", "CINEPOLIS"],
    "Compras y retail": ["AMAZON", "AMAZON PRIME", "MERCADOPAGO", "MERCADO PAGO", "LIVERPOOL", "SAMS CLUB", "SEARS", "SHEIN", "TEMU", "COPPEL"],
    "Servicios financieros y pagos": ["KUESKI", "KUESKI PAY", "APLAZO", "CASHI", "TELCEL", "GOOGLE PAY", "APPLE PAY", "ALLIANZ", "BANAMEX", "SOFT RAPPI", "MI ATT", "ATT"],
    "Tecnología y software": ["MICROSOFT", "ADOBE", "GOOGLE", "GOOGLE ONE", "CANVA", "OPENAI"],
    "Salud y farmacia": ["FARMACIAS DEL AHORRO", "FARMACIAS GUADALAJARA", "ALSUPER", "FARMACIAS SIMILARES"],
    "Gasto en hogar": ["IZZI", "TOTALPLAY", "MEGACABLE", "TELMEX", "CFE", "ROTOPLAS"],
    "Educación y cultura": ["LIBRERIAS", "EDUCACION", "CURSOS", "SMART FIT"],
}

## 🗂️ 2. Preparación temporal del dataset

Se convierte la columna de fechas y se genera una nueva columna de año-mes (año_mes) para facilitar la agrupación.

In [2]:
df['fecha'] = pd.to_datetime(df['fecha'], dayfirst=True)
df['año_mes'] = df['fecha'].dt.to_period('M')

## 📉 3. Generación de resumen mensual

Agrupamos por cliente (id), comercio y mes, obteniendo el total de gasto y número de transacciones.

In [3]:
resumen_mensual = df.groupby(['id', 'comercio', 'año_mes']).agg(
    monto_total=('monto', 'sum'),
    transacciones_mes=('monto', 'count')
).reset_index()

ultimo_mes = df['año_mes'].max()  # Último mes del dataset

## 🔁 4. Detección de patrones recurrentes

Se analiza la secuencia de gastos por cliente y comercio para detectar patrones mensuales o bimestrales consistentes antes del ultimo_mes, y se estima si habrá una nueva transacción en ese último mes.

In [6]:
predicciones = []

for (cliente, comercio), grupo_original in resumen_mensual.groupby(['id', 'comercio']):
    grupo = grupo_original[grupo_original['año_mes'] < ultimo_mes].copy()
    grupo = grupo.sort_values('año_mes').reset_index(drop=True)

    if len(grupo) < 4:
        continue

    meses_dt = pd.Series(pd.PeriodIndex(grupo['año_mes'], freq='M'))
    difs = meses_dt.diff().dropna().apply(lambda x: x.n)

    for tipo, nombre_patron in [(1, 'mensual'), (2, 'bimestral')]:
        indices_patron = difs[difs == tipo].index

        # Buscar secuencias
        secuencias = []
        sec_actual = []
        for idx in indices_patron:
            if not sec_actual or idx == sec_actual[-1] + 1:
                sec_actual.append(idx)
            else:
                if len(sec_actual) >= 2:
                    secuencias.append(sec_actual)
                sec_actual = [idx]
        if len(sec_actual) >= 2:
            secuencias.append(sec_actual)

        if not secuencias:
            continue

        secuencia_larga = max(secuencias, key=len)
        meses_validos = grupo.loc[secuencia_larga[0]:secuencia_larga[-1]+1]

        ultimos_3 = [ultimo_mes - i for i in range(1, 4)]
        presentes = meses_validos['año_mes'].isin(ultimos_3).sum()
        if presentes < 3:
            continue

        monto_promedio = meses_validos['monto_total'].mean()
        transacciones_prom = meses_validos['transacciones_mes'].mean()
        meses_de_recurrencia = len(meses_validos)

        año, mes = ultimo_mes.year, ultimo_mes.month
        dia_valido = min(5, calendar.monthrange(año, mes)[1])
        fecha_estimado = datetime(año, mes, dia_valido)

        predicciones.append({
            'id': cliente,
            'comercio': comercio,
            'patron': nombre_patron,
            'fecha_estimado': fecha_estimado,
            'monto_estimado': round(monto_promedio, 2),
            'promedio_transacciones_mes': round(transacciones_prom, 2),
            'meses_de_recurrencia': meses_de_recurrencia
        })

df_predicciones = pd.DataFrame(predicciones)
df_predicciones

,id,comercio,patron,fecha_estimado,monto_estimado,promedio_transacciones_mes,meses_de_recurrencia
0,003d9abe467a91847d566cf455bd2d7d6c8f7e75,AMAZON PRIME,mensual,2023-01-05,11.62,1.00,11
1,003d9abe467a91847d566cf455bd2d7d6c8f7e75,DIDI RIDES,mensual,2023-01-05,87.42,10.18,11
2,003d9abe467a91847d566cf455bd2d7d6c8f7e75,MERCADO PAGO,mensual,2023-01-05,51.05,1.86,7
3,003d9abe467a91847d566cf455bd2d7d6c8f7e75,SPOTIFY,mensual,2023-01-05,23.10,1.00,11
4,003d9abe467a91847d566cf455bd2d7d6c8f7e75,UBER,mensual,2023-01-05,257.08,31.73,11
...,...,...,...,...,...,...,...
5213,fe9415c62193f2d430a0340c31064ec512b27c8c,TELMEX,mensual,2023-01-05,69.05,1.00,11
5214,ff67da037fae796809be0e36fb9cdd0e191c38a4,DISNEY PLUS,mensual,2023-01-05,34.59,1.00,5
5215,ff67da037fae796809be0e36fb9cdd0e191c38a4,GOOGLE YOUTUBEPREMIUM,mensual,2023-01-05,32.29,1.00,11
5216,ff67da037fae796809be0e36fb9cdd0e191c38a4,MEGACABLE,mensual,2023-01-05,158.14,1.82,11


## ✅ 5. Validación: cálculo del porcentaje de aciertos

Se compara si para cada predicción hubo efectivamente una transacción en ultimo_mes, para evaluar la precisión del modelo.

In [5]:
# Transacciones reales del último mes
reales_ultimo_mes = resumen_mensual[resumen_mensual['año_mes'] == ultimo_mes]

# Crear claves para facilitar comparación
df_predicciones['clave'] = df_predicciones['id'].astype(str) + '___' + df_predicciones['comercio']
reales_ultimo_mes['clave'] = reales_ultimo_mes['id'].astype(str) + '___' + reales_ultimo_mes['comercio']

# Comparación
predicciones_acertadas = df_predicciones['clave'].isin(reales_ultimo_mes['clave']).sum()
total_predicciones = len(df_predicciones)

# Porcentaje de acierto
porcentaje_acierto = (predicciones_acertadas / total_predicciones) * 100 if total_predicciones > 0 else 0

print(f"Predicciones acertadas: {predicciones_acertadas} de {total_predicciones}")
print(f"Porcentaje de aciertos: {porcentaje_acierto:.2f}%")

Predicciones acertadas: 4702 de 5218
Porcentaje de aciertos: 90.11%


/var/folders/6h/nf6jxxkx3c98vxmkgwqt8jzw0000gn/T/ipykernel_5077/3383190450.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reales_ultimo_mes['clave'] = reales_ultimo_mes['id'].astype(str) + '___' + reales_ultimo_mes['comercio']
